In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(12_000_000_000_000)

import pandas as pd
from siuba import *

from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.tables import tbls
import shared_utils
import datetime as dt

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/cloud/bigquery/table.py:43: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/

In [2]:
analysis_date = dt.date(2023, 3, 15)

In [3]:
bbb_ix_df = shared_utils.rt_utils.get_speedmaps_ix_df(analysis_date, 300)

In [4]:
bbb_ix_df

,feed_key,schedule_gtfs_dataset_key,vehicle_positions_gtfs_dataset_key,organization_itp_id,organization_name,activity_date
0,4f77ef02b983eccc0869c7540f98a7d0,dbbe8ee4864a2715a40749605395d584,6c2d7daaf979779fa2089c6395baf98b,300,City of Santa Monica,2023-03-15


In [5]:
bbb_base64 = 'aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZGF0ZXMuYmlu'

In [46]:
bbb_trips = shared_utils.rt_utils.get_trips(bbb_ix_df)

found trips parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/trips_300_2023-03-15.parquet


In [49]:
bbb_st = shared_utils.rt_utils.get_st(bbb_ix_df, bbb_trips)

found stop times parquet at gs://calitp-analytics-data/data-analyses/rt_delay/v2_cached_views/st_300_2023-03-15.parquet


In [57]:
bbb_st.arrival_time.iloc[0].split(':')

['06', '26', '02']

In [61]:
shared_utils.rt_utils.show_full_df(bbb_st >> group_by(_.trip_id) >> summarize(min_arrival = _.arrival_time.min()))

,trip_id,min_arrival
0,893044,06:59:00
1,893045,07:10:00
2,893046,07:19:00
3,893047,07:29:00
4,893048,07:40:00
5,893049,07:49:00
6,893050,07:59:00
7,893051,08:10:00
8,893052,08:20:00
9,893053,08:30:00


In [25]:
# df = (
#     tbls.mart_ad_hoc.fct_stop_time_updates_20230315_to_20230321()
#     >> filter(_.service_date == analysis_date,
#               # _.trip_id == '894836',
#               _.arrival_time_pacific >= pac_ts,
#               _.arrival_time_pacific < pac_ts2,
#              _.base64_url == bbb_base64)
#     >> select(_.arrival_time_pacific, _.departure_time_pacific,
#              _.key, _.gtfs_dataset_key, _.base64_url,
#               _._extract_ts, _.trip_update_timestamp, _.trip_id,
#              _.stop_sequence, _.stop_id, _.service_date)
#     >> collect()
# )

In [12]:
df.base64_url.iloc[0]

'aHR0cDovL2d0ZnMuYmlnYmx1ZWJ1cy5jb20vdHJpcHVwZGF0ZXMuYmlu'

In [21]:
df.columns

Index(['key', 'gtfs_dataset_key', 'dt', 'hour', 'base64_url', '_extract_ts',
       '_config_extract_ts', '_gtfs_dataset_name', '_header_message_age',
       '_trip_update_message_age', '_trip_update_message_age_vs_header',
       'header_timestamp', 'header_version', 'header_incrementality', 'id',
       'trip_update_timestamp', 'trip_update_delay', 'vehicle_id',
       'vehicle_label', 'vehicle_license_plate', 'trip_id', 'trip_route_id',
       'trip_direction_id', 'trip_start_time', 'trip_start_date',
       'trip_schedule_relationship', '_trip_updates_message_key',
       'stop_sequence', 'stop_id', 'arrival_delay', 'arrival_time',
       'arrival_uncertainty', 'departure_delay', 'departure_time',
       'departure_uncertainty', 'schedule_relationship', 'service_date',
       'arrival_time_pacific', 'departure_time_pacific'],
      dtype='object')

In [24]:
df.to_parquet('bbb_example.parquet')

In [29]:
df.stop_sequence.isna().value_counts()

False    684351
Name: stop_sequence, dtype: int64

In [36]:
filtered._extract_ts.iloc[0]

Timestamp('2023-03-15 14:43:20+0000', tz='UTC')

In [37]:
import pytz

In [40]:
pacific = pytz.timezone('US/Pacific')

In [62]:
filtered = df >> filter(_.trip_id == '893051', _.stop_sequence == 2) >> arrange(_._extract_ts)
filtered = filtered >> mutate(arrival_time_pacific = _.arrival_time_pacific.apply(lambda x: pacific.localize(x)))
filtered = filtered >> mutate(time_in_advance = _.arrival_time_pacific - _._extract_ts)
filtered = filtered >> select(-_.key, -_.gtfs_dataset_key, -_.base64_url)

In [ ]:
## yep, this is what we wanna see!
## can do a tighter arrival time filter =-- 30min periods??

In [65]:
shared_utils.rt_utils.show_full_df(filtered)

,arrival_time_pacific,departure_time_pacific,_extract_ts,trip_update_timestamp,trip_id,stop_sequence,stop_id,service_date,time_in_advance
276780,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:10:40+00:00,2023-03-15 14:10:30+00:00,893051,2,361,2023-03-15,0 days 01:00:14
277242,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:11:00+00:00,2023-03-15 14:10:50+00:00,893051,2,361,2023-03-15,0 days 00:59:54
277661,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:11:20+00:00,2023-03-15 14:11:06+00:00,893051,2,361,2023-03-15,0 days 00:59:34
275258,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:11:40+00:00,2023-03-15 14:11:30+00:00,893051,2,361,2023-03-15,0 days 00:59:14
275680,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:12:00+00:00,2023-03-15 14:11:50+00:00,893051,2,361,2023-03-15,0 days 00:58:54
276279,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:12:20+00:00,2023-03-15 14:12:06+00:00,893051,2,361,2023-03-15,0 days 00:58:34
276960,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:12:40+00:00,2023-03-15 14:12:30+00:00,893051,2,361,2023-03-15,0 days 00:58:14
277484,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:13:00+00:00,2023-03-15 14:12:50+00:00,893051,2,361,2023-03-15,0 days 00:57:54
278639,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:13:20+00:00,2023-03-15 14:13:11+00:00,893051,2,361,2023-03-15,0 days 00:57:34
275418,2023-03-15 08:10:54-07:00,NaT,2023-03-15 14:13:40+00:00,2023-03-15 14:13:30+00:00,893051,2,361,2023-03-15,0 days 00:57:14


In [ ]:
## sampling approach: use 8a, noon, 11p for all operators?

In [9]:
pac_ts = dt.datetime.combine(analysis_date, dt.time(8, 0))

In [14]:
pac_ts_sec = int(pac_ts.timestamp())

In [15]:
pac_ts2 = pac_ts + dt.timedelta(hours = 1)

In [16]:
pac_ts2_sec = int(pac_ts2.timestamp())